In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
dataset_path = 'Phishing_Email.csv'
df = pd.read_csv(dataset_path)
emails = df['Email Text'].astype(str).values
labels = (df['Email Type'] == 'Safe Email').astype(int).values

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(emails)
sequences = tokenizer.texts_to_sequences(emails)
data = pad_sequences(sequences, maxlen=500)

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
model.add(Embedding(10000, 128, input_length=500))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
373/373 [==============================] - 43s 113ms/step - loss: 0.2990 - acc: 0.8530 - val_loss: 0.1528 - val_acc: 0.9410
Epoch 2/10
373/373 [==============================] - 41s 109ms/step - loss: 0.1088 - acc: 0.9553 - val_loss: 0.1453 - val_acc: 0.9460
Epoch 3/10
373/373 [==============================] - 41s 111ms/step - loss: 0.0793 - acc: 0.9642 - val_loss: 0.1527 - val_acc: 0.9494
Epoch 4/10
373/373 [==============================] - 42s 112ms/step - loss: 0.0702 - acc: 0.9680 - val_loss: 0.1672 - val_acc: 0.9521
Epoch 5/10
373/373 [==============================] - 40s 106ms/step - loss: 0.0610 - acc: 0.9708 - val_loss: 0.1879 - val_acc: 0.9474
Epoch 6/10
373/373 [==============================] - 40s 107ms/step - loss: 0.0563 - acc: 0.9727 - val_loss: 0.1839 - val_acc: 0.9467
Epoch 7/10
373/373 [==============================] - 42s 114ms/step - loss: 0.0513 - acc: 0.9739 - val_loss: 0.1949 - val_acc: 0.9484
Epoch 8/10
373/373 [==============================] - 4

In [ ]:
import numpy as np

def predict_phishing(email):

    sequence = tokenizer.texts_to_sequences([email])

    padded_sequence = pad_sequences(sequence, maxlen=500)

    prediction = model.predict(np.array(padded_sequence))

    return 'Phishing' if prediction[0][0] > 0.5 else 'Not Phishing'

# Example usage:
new_email = "Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/123456 tp claim now."
result = predict_phishing(new_email)
print(f'The email is predicted as: {result}')

1/1 [==============================] - 0s 113ms/step
The email is predicted as: Phishing


In [ ]:
new_email = "Hi there, yes! We just released the newest version of those shoes yesterday. (Skyrider V2.) Here’s the link >> www.shoes.SkyriderV2.com — Tommy, Customer Service Representative @ AllPath Shoes"
result = predict_phishing(new_email)
print(f'The email is predicted as: {result}')

1/1 [==============================] - 0s 21ms/step
The email is predicted as: Phishing


In [ ]:
new_email = "hi [mohammed],We recently detected an unusual login attempt to your account. To ensure the security of your account, we recommend that you review the details below:- **Date and Time:** [3/4/2024]- **Location:** [158.142.32.145]- **Device:** [Asus TUF Gaming]If you recognize this activity, you can ignore this email. However, if you did not initiate this login, please take the following steps immediately:1. Change your password by visiting our secure website: [Link to Password Reset]2. Enable two-factor authentication (2FA) for an added layer of security.Thank you for helping us keep your account safe.Best regards,[google] Security Team"
result = predict_phishing(new_email)
print(f'The email is predicted as: {result}')

1/1 [==============================] - 0s 20ms/step
The email is predicted as: Not Phishing


In [ ]:
new_email = "We couldn’t process the October Netflix monthly payment, and the account is on hold, temporarily. Please update your billing information at: billingproblem0477.in to avoid any restrictions to your account and full access to Netflix.IGNODYFS70Y8 NET-MEMBER.7:46 AM"
result = predict_phishing(new_email)
print(f'The email is predicted as: {result}')

1/1 [==============================] - 0s 21ms/step
The email is predicted as: Phishing


In [ ]:
new_email = "Deer subscription ,Amazon is sending you a refund of $32.64. Please reply with your bank account and routing number to receive your refund"
result = predict_phishing(new_email)
print(f'The email is predicted as: {result}')

1/1 [==============================] - 0s 26ms/step
The email is predicted as: Phishing


In [ ]:
new_email = "Hi Mohammed Nemr (@mohammednemr),We noticed a new login to your HF account. Here are the details:Location: Syria (Approximate location)Time: 30 May 2024 07:14 AM..AM UTC If you logged in to your account recently, there's nothing for you to do right now.If you don't recognize this login, we recommend you change your password to a strong one you haven't used before by clicking here. You can also reply to this email directly if you need further assistance.Cheers,The Hugging Face team"
result = predict_phishing(new_email)
print(f'The email is predicted as: {result}')

1/1 [==============================] - 0s 20ms/step
The email is predicted as: Not Phishing


In [ ]:
!pip install waitress
!pip install flask
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    email = request.json['email']
    result = predict_phishing(email)
    return jsonify({'prediction': result})

if __name__ == '__main__':
    from waitress import serve
    serve(app, host="0.0.0.0", port=8081)
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install streamlit
import streamlit as st

st.title('Phishing Email Prediction')

email = st.text_input("Enter the email text:")

if st.button('Predict'):
    result = predict_phishing(email)
    st.write(f'The email is predicted as: {result}')
